In [ ]:
from datascience import *
from scipy.stats import norm
from IPython.display import clear_output
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use("fivethirtyeight")

# DSC 10 Discussion Week 8
---

# Confidence Intervals
---

Tonight, we'll be collecting data from across the world and calculating confidence intervals—fun!

In [ ]:
full_data = Table.read_table("Life Expectancy Data.csv")

In [ ]:
full_data

This data comes from the World Health Organization.  We can learn more about the meanings of the columns by looking here: https://www.kaggle.com/kumarajarshi/life-expectancy-who

Let's travel back in time to the year 2015 and collect some data!  For the duration of this discussion, we're going to consider the following data to be our *"population"*.

Let's take a look at it.

In [ ]:
recent_data = full_data.where("Year", 2015)

In [ ]:
recent_data

In [ ]:
# Let's visualize our population distribution.

# Defining a function to create bins easily
def get_bins(array, bin_size=1):
    return np.arange(
        int(min(array)), int(max(array)) + 1 + bin_size, bin_size
    )

In [ ]:
MEASURE = "Life expectancy "
data = recent_data.where(MEASURE, are.above_or_equal_to(0)) # This is just to remove nans
measured = data.column(MEASURE)
bins = get_bins(measured, 5) # <-- Try playing around with the bin size

data.hist(MEASURE, bins=bins)

In [ ]:
data.sort("Life expectancy ", descending=True)

# This is our ... ? (Population or Sample Distribution)

So, what is our aim?  We want to estimate the average life expectancy for the globe!  Let's say we don't have access to the entire population.  Flying around the world is pretty expensive, so we can only collect data from 15 countries

As are standard procedures, we'll be creating a confidence interval around this estimate.  We can sample and use bootstrapping to find this.

In [ ]:
# How do we create a representative sample?
collected = data.sample(15, with_replacement=False)

In [ ]:
collected

In [ ]:
data.hist(MEASURE, bins=bins)

In [ ]:
collected.hist(MEASURE, bins=get_bins(collected.column(MEASURE), 5))

# This is our ...? (Sample or Population Distribution)

We're interested in estimating the mean life expectancy.  So, let's find the mean of our sample.

In [ ]:
sample_mean = np.mean(collected.column(MEASURE))
sample_mean

In [ ]:
# We can show our mean in relation to the sample.

collected.hist(MEASURE, bins=get_bins(collected.column(MEASURE), 5))
plt.axvline(sample_mean, c='r');

What happens when we resample?

In [ ]:
# Run this multiple time to see what changes.

resampled = collected.sample()
resampled_mean = np.mean(resampled.column(MEASURE))

print("The resampled mean is:\t\t", resampled_mean, "\nCompared to the original:\t", sample_mean)

resampled.hist(MEASURE, bins=get_bins(collected.column(MEASURE),5))
plt.axvline(resampled_mean, c='r')
plt.axvline(sample_mean, c='b')
plt.legend(["Resampled Mean","Sample Mean"]);

# What do the red and blue lines represent again?

```
RED: RESAMPLED MEAN
BLUE: SAMPLE MEAN
```

Now, let's run the bootstrap so we can create our confidence interval!

In [ ]:
sample_means = []

for i in range(10000):
    bootstrapped = collected.sample()
    boot_mean = np.mean(bootstrapped.column(MEASURE))
    sample_means.append(boot_mean)

plt.hist(sample_means, bins=get_bins(sample_means, 0.5));

In [ ]:
# super fast version ~woah~
n = collected.num_rows
life_exp = collected.column(MEASURE)
sample_means = np.array([life_exp[np.random.choice(n, n)].mean() for i in range(10000)])

plt.hist(sample_means, bins=get_bins(sample_means, 0.5));

# What does this graph represent?

Recall, the confidence interval uses percentiles as the bounds.

In [ ]:
# First, we choose a confidence level.
confidence_level = 95
 
# What is the mean we're estimating?
mean = np.mean(sample_means)

# And then our lower and upper bounds?
# Let's try to find a way to calculate them regardless of our confidence level.
lower_bound = percentile((100 - confidence_level) / 2, sample_means)
upper_bound = percentile(confidence_level + (100 - confidence_level) / 2, sample_means)

# Printing it out so we can easily see our results.
print("""
 Mean:\t{}

Lower:\t{}
Upper:\t{}

Level:\t{}%
""".format(mean, lower_bound, upper_bound, confidence_level))

Out of curiosity... what's the difference between our bounds and the mean?

In [ ]:
mean-lower_bound

In [ ]:
upper_bound-mean

# How do we interpret this confidence interval?
---

# Let's get a feel for how the distribution of sample means arises
---

We'll take a bunch of resamples and visualize the distribution as the number of resamples increases.

In [ ]:
trials = 10000

sample_means = []

update_var = 1

for i in range(trials):
    boots = collected.sample()
    boot_mean = np.mean(boots.column(MEASURE))
    sample_means.append(boot_mean)
    
    # Update the plot every once in a while (more frequently at lower values)
    if i >= update_var - 1 or i==trials - 1:
        update_var *= 1.25
        clear_output(wait=True)
        
        plt.hist(sample_means, density=True, bins=get_bins(sample_means, 0.5))
        plt.axvline(np.mean(sample_means), c='r')
        
        plt.pause(0.01)

        print("Resamples:\t",i + 1)

# A bit of recap
---

Let's draw some things on the chalkboard.

- Our **POPULATION DISTRIBUTION** is unknown, and can be any shape.


- A **SAMPLE DISTRIBUTION** should have a shape roughly similar to the population distribution.  
(provided that the sample was large enough and was properly randomized)


- A **SAMPLE MEAN** is just the mean of that sample distribution.


- We can collect a handful of sample means (or fake it by bootstrapping)


- The **DISTRIBUTION OF SAMPLE MEANS** will resemble a normal distribution as the number of sample means increases. (Law of Large Numbers)


- The **CENTER/MEAN** of the distribution of sample means should be similar to the true population mean.  
(provided that our original sample was proper)

In [ ]:
# Let's try this out on another column to see if the above claims hold!

# What if I told you there's another way?
---

Since we know that a normal distribution will arise as the number of resamples increases, then do we really need to go through all the effort of running a bootstrap?

Instead, we can rely on what we know about normal distributions!  The two defining features of a normal distribution are its center/mean and it's spread/standard deviation.

So, what's the standard deviation of the normal distribution that arises?

In [ ]:
# Running a bootstrap again to get the distribution
m = collected.column(MEASURE)
resampled_means = [m[np.random.choice(n,n)].mean() for i in range(10000)]
plt.hist(resampled_means, bins=get_bins(resampled_means, 0.5), density=True)

# Plotting a normal curve over the top of it.
mean = np.mean(collected.column(MEASURE)) # The mean should be ..?

# Let's figure out what the standard deviation of the sample means should be!
std = np.std(collected.column(MEASURE)) / np.sqrt(collected.num_rows)
# Well, it SHOULD be the population SD / sqrt(Sample Size)
# but we can't access the population sd!  So, we use the sample sd instead.

x = np.linspace(*plt.xlim(), 100)
plt.plot(x, norm.pdf(x, mean, std), c='r');

So, we can create our CI mathemagically instead of running the bootstrap!

For our normal curve, we'll have:
$$Mean = Sample\ Mean$$

$$Standard\ Deviation\ of\ Sample\ Means = \frac{Sample\ SD}{\sqrt{Sample\ Size}}$$

Now that we have a normal curve with a mean and standard deviation, how do we find the confidence interval from before?

In [ ]:
# We know that 95% of data is within 1.96 standard deviations of
# the mean of a normal distribution.

mean = np.mean(collected.column(MEASURE))

# What's the SD of the distrib. of sample means again?
std = np.std(collected.column(MEASURE)) / np.sqrt(collected.num_rows)

lower_bound = mean - 1.96 * std
upper_bound = mean + 1.96 * std

print("""
 Mean:\t{}

Lower:\t{}
Upper:\t{}

Level:\t95%
""".format(mean, lower_bound, upper_bound))

Just like before, I'm curious to see the difference between our bounds and mean.

In [ ]:
mean-lower_bound

In [ ]:
upper_bound-mean

In [ ]:
# How close were we to our bootstrap confidence interval?

In [ ]:
# What happens if we decrease/increase the number of resamples?

Cool!  Now, since we do have the *"population"* that we're estimating, let's check the validity of our confidence interval.

In [ ]:
# What was the population mean?
pop_mean = np.mean(data.column(MEASURE))
pop_mean

In [ ]:
# Is the population mean inside of our interval?
lower_bound <= pop_mean <= upper_bound

# Interval widths
---

What if we think our interval size is too big?  How does our interval size change as other parameters change?

# Provided that we want to keep 95% level of confidence, how will we narrow our interval width?

In [ ]:
# BEFORE we answer that, let's think about one more thing:
#
# What is the width of our interval, in terms of our mean and standard deviation?


$$Interval\ Width = 4\cdot Sample\ Mean\ Standard\ Deviation$$
$$= 4\cdot \frac{Sample\ SD}{\sqrt{Sample\ Size}}$$

# Whats potentially wrong with what we've done above?

In [ ]:
# What happens if we can't tolerate a 5% chance of being wrong?
#
# If this is really important, we may want 99.7% confidence!
#
# What will happen to the width of our interval?

# What will our interval width be in term of mean and standard deviation?

To answer this, we need to know the "Empirical Rule".

It goes a little something like this:

|# of SD away from mean|% of data contained in ± #SD|
|---|---|
|1|68%|
|2|95%|
|3|99.7%|

In [ ]:
# So, we can calculate the CI if we need 99.7% confidence

mean = np.mean(collected.column(MEASURE))

std = np.std(collected.column(MEASURE)) / np.sqrt(collected.num_rows)

lower_bound_997 = mean - 3 * std
upper_bound_997 = mean + 3 * std

print("""
 Mean:\t{}

Lower:\t{}
Upper:\t{}

Level:\t99.7%
""".format(mean, lower_bound_997, upper_bound_997))

In [ ]:
# Let's look at the bounds for 95% versus 99.7% confidence
print("""
95% Lower:\t{0}\t99.7% Lower:\t{2}
95% Upper:\t{1}\t99.7% Upper:\t{3}
""".format(lower_bound, upper_bound, lower_bound_997, upper_bound_997))